In [ ]:
import SpaDecon as spd
import scanpy as sc
import pandas as pd
import numpy as np
from skimage import io
import os

In [ ]:
#Please download the spadecon_tutorial_data folder from https://drive.google.com/drive/folders/1_eBGKMVYh4p1eiHhrJASnqOD9z_CP5e0?usp=sharing

#set working directory to spadecon_tutorial_data using os.chdir()

#The next two code chunks give the same results; only run one of them depending on data file preference
os.chdir('/Users/kylepcoleman/Downloads/spadecon_tutorial_data/')

In [ ]:
#Read in scRNA-seq/SRT GE data using h5/h5ad files

#Read annotated scRNA-seq GE data (rows = cells, columns = genes, cell types in adata_sc.obs.celltype)
adata_sc = sc.read('bc_sc.h5ad')

#Read SRT GE data (rows = spots, columns = genes)
adata_srt = sc.read_10x_h5('V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5')
adata_srt.var_names_make_unique()

In [ ]:
#Read in scRNA-seq/SRT GE data using csv files

#Read scRNA-seq GE data (rows = cells, columns = genes)
sc_ge = pd.read_csv('bc_sc_ge.csv', index_col = 0)
  
#Read scRNA-seq cell-type labels
sc_types = pd.read_csv('bc_sc_types.csv', index_col = 0)
  
#Convert scRNA-seq GE data to AnnData object
adata_sc = sc.AnnData(sc_ge)
  
#Insert cell-type labels into "celltype" column adata_sc.obs
adata_sc.obs['celltype'] = sc_types['celltype']
  
#Read SRT GE data (rows = cells, columns = genes)
srt_ge = pd.read_csv('bc_srt_ge.csv', index_col = 0)
  
#Convert SRT GE data to AnnData object
adata_srt = sc.AnnData(srt_ge)

In [ ]:
#Read SRT histology image
histology = io.imread("V1_Breast_Cancer_Block_A_Section_1_image.tif")

#Read file with SRT spatial locations
locations = pd.read_csv('tissue_positions_list.csv', header = None, index_col = 0)
locations = locations.loc[adata_srt.obs.index]

In [ ]:
#Run SpaDecon

np.random.seed(10)
clf = spd.SpaDecon()
#The technology parameter is used to determine an upper bound on the number of cell types per spot (specify "Visium" or "ST")
clf.deconvolution(source_data=adata_sc, target_data=adata_srt, histology_image=histology, spatial_locations=locations, technology = 'Visium')

In [ ]:
spadecon_props = clf.props
spadecon_props.to_csv('spadecon_props.csv')